# 🧹 01 - Prétraitement des tweets
Ce notebook a pour objectif de charger, nettoyer et sauvegarder un sous-ensemble des tweets en vue de l'entraînement des modèles.

In [ ]:
import pandas as pd
import numpy as np
import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
import nltk

## 📥 Chargement des données

In [ ]:
df = pd.read_csv("../data/raw/training.1600000.processed.noemoticon.csv", encoding="ISO-8859-1", header=None)
df.columns = ['sentiment', 'id', 'date', 'query', 'user', 'text']
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1})  # 0 = négatif, 1 = positif

df.sample(5)

## 📊 Répartition des classes

In [ ]:
sns.countplot(data=df, x='sentiment')
plt.title("Répartition des tweets par sentiment")
plt.xticks([0,1], ['Négatif', 'Positif'])
plt.show()

## 🧹 Nettoyage des textes

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # liens
    text = re.sub(r'\@\w+|\#','', text)  # mentions & hashtags
    text = re.sub(r'[^\w\s]', '', text)  # ponctuation
    text = re.sub(r'\d+', '', text)  # chiffres
    text = re.sub(r'\s+', ' ', text).strip()  # espaces
    return text

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

df['clean_text'] = df['text'].apply(clean_text).apply(remove_stopwords)
df[['text', 'clean_text']].sample(5)

## 💾 Sauvegarde d’un sous-ensemble équilibré

In [ ]:
df_balanced = (
    df.groupby("sentiment")
      .apply(lambda x: x.sample(n=50000, random_state=42))
      .reset_index(drop=True)
)

df_balanced.to_csv("../data/processed/tweets_clean.csv", index=False)
print("✅ Données nettoyées et enregistrées dans data/processed/tweets_clean.csv")